In [25]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import datetime as dt

In [14]:
# Define the basket's tickers and weights
tickers = ['BHP.AX', 'CSL.AX', 'WDS.AX', 'MQG.AX']
weights = np.array([0.1, 0.35, 0.15, 0.4])

In [15]:
# Extracting closing prices as of Friday, 16 May 2025 from yfinance
data = yf.download(tickers, start='05-16-2025', end='05-17-2025')['Close']

# Extract the closing prices for that date
closing_prices = data.loc['05-16-2025'].values

# Calculate weighted average (basket spot price S0)
S0 = np.dot(weights, closing_prices)

print("Basket spot price S0 as of 16 May 2025:", S0)

[*********************100%***********************]  3 of 4 completed

4 Failed downloads:
['BHP.AX', 'WDS.AX', 'CSL.AX', 'MQG.AX']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


KeyError: '05-16-2025'

In [34]:
# Define the basket's tickers and weights
tickers = [50.53, 60.20, 45.10, 70.25]
weights = np.array([0.1, 0.35, 0.15, 0.4])

# Download latest closing prices
S0 = np.dot(weights, tickers)

print("Basket spot price S0:", S0)

K = 75
valuation_date = dt. datetime(2025, 5, 16)
expiry_date = dt.datetime(2025, 7, 17)
T = (expiry_date - valuation_date).days / 365
sigma = 0.5  # Replace with your basket's volatility estimate
N = 100

price, stock_tree, option_tree = binomial_european_call(S0, K, T, sigma, N)
print(f"European call basket option price: {price:.2f} AUD")

Basket spot price S0: 60.988
dt: 0.0016986301369863012
u: 1.0208210152536263
d: 0.979603657308668
p: 0.4948483771950943
European call basket option price: 1.14 AUD


In [16]:
def binomial_european_call(S0, K, T, sigma, N):
    """Calculate European call option price using binomial model"""
    # Calculate time step
    dt = T/N
    print(f"dt: {dt}")
    
    # Calculate up and down factors
    u = np.exp(sigma * np.sqrt(dt))
    print(f"u: {u}")
    d = 1/u
    print(f"d: {d}")
    
    # Calculate risk-neutral probability
    p = 1 / (u + 1)
    print(f"p: {p}")
    
    # Initialize stock price tree
    stock_tree = np.zeros((N+1, N+1))
    
    # Populate the stock price tree through time
    for i in range(N+1):
        for j in range(i+1):
            stock_tree[i, j] = S0 * (u ** j) * (d ** (i - j))
    
    # Initialize option value tree
    option_tree = np.zeros((N+1, N+1))
    
    # Calculate call option payoffs at expiration
    for j in range(N+1):
        option_tree[N, j] = max(0, stock_tree[N, j] - K)
    
    # Work backwards discounting and reflecting risk-neutral probabilities to calculate option value at each node
    for i in range(N-1, -1, -1):
        for j in range(i+1):
            option_tree[i, j] = (p * option_tree[i+1, j+1] + (1-p) * option_tree[i+1, j])
    
    return option_tree[0, 0], stock_tree, option_tree